In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import scipy.io as sio
from dataclasses import dataclass
from typing import List, Tuple
import os
from dotenv import load_dotenv
load_dotenv()
import tidy3d as td
from tidy3d import web
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from natsort import natsorted
import numpy as np
import matplotlib.animation as animation
import xarray as xr
import imageio
import matplotlib
import sys
import os

# Assuming /AutomationModule is in the root directory of your project
sys.path.append(os.path.abspath(r'H:\phd stuff\tidy3d'))

from AutomationModule import * 

import AutomationModule as AM

In [2]:
tidy3dAPI = os.environ["API_TIDY3D_KEY"]

In [3]:
a = 1
medium = td.Medium(permittivity=11.56)
run = True

In [4]:
# lambdas =  a/np.array([0.24,0.44])
# lambdas =  a/np.array([0.14,0.18])
lambdas =  a/np.array([0.14,0.18])

In [5]:
folder_path=rf"H:\phd stuff\tidy3d\structures\2D SHU"
sample = 3
project_name = fr"10_02_2024 Continuous Wave Periodic Conditions L=15"
empty=False
runtime_ps = 10e-12
min_steps_per_lambda = 20
id0 = ""
add_ref = True

In [ ]:
for l,file in enumerate(natsorted(os.listdir(folder_path))):
    print(Path(file).stem)
    data = []
    centers = []
    slicing= 0.15
    if l !=6:
        continue

    with open(os.path.join(folder_path, file),"r") as f: 
        lines = f.read().splitlines() 
        i=0
        index = 1
        num_samples = int(lines[index].split()[0])

        while num_samples>0 and i < 300:
            try:
                num_samples = int(lines[index].split()[0])
                info = lines[index]
                old_index = index
                index += num_samples+1
                li = (lines[old_index+1:index])
                centers = np.array([list(map(float, item.split())) for item in li])
                data += [{
                    "centers":centers,
                    "Lx":float(lines[old_index].split()[1]),
                    "Ly":float(lines[old_index].split()[2]),
                    "points":float(lines[old_index].split()[0]),
                }]
                i+=1
            except:
                num_samples = 0
    

    for k,item in enumerate(data):
        #Tight pulse 

        if k != sample:
            continue

        run_name = f"{Path(file).stem} Beam Spreading {1/lambdas[0]:.3g} - {1/lambdas[1]:.3g} - Sample_{k} {runtime_ps*1e12}ps"
        print(run_name)
        structure_1 = AM.loadAndRunStructure(key = tidy3dAPI
                    ,direction="z", lambda_range=lambdas,
                    box_size= data[k]['Lx']*a,runtime_ps=runtime_ps,min_steps_per_lambda=min_steps_per_lambda,
                   scaling=1,shuoff_condtion=1e-20, verbose=True,
                   monitors=["flux"],
                   freqs=250, 
                   source="planewave", 
                   width=0.35, ref_only=True
                   )


        sim = structure_1.sim



        boundaries= td.BoundarySpec(
                # x=td.Boundary(plus=td.Absorber(num_layers=500),minus=td.Absorber(num_layers=500)),
                y=td.Boundary(plus=td.Absorber(num_layers=500),minus=td.Absorber(num_layers=500)),
                x=td.Boundary.periodic(),
                z=td.Boundary.periodic(),
            )
        sim = sim.copy(update={'boundary_spec':boundaries})
        sim = sim.copy(update={'sources':[]})
        sim = sim.copy(update={'monitors':[]})


        Lx, Ly = data[k]['Lx'],data[k]['Ly']*slicing+40
        sim = sim.copy(update={'size':[Lx,Ly,0]})

        source = td.PlaneWave(
                source_time = td.ContinuousWave(
                    freq0=structure_1.freq0,
                    fwidth=structure_1.freqw
                ),
                size= (td.inf,0,td.inf),
                center=(0,-(Lx)*slicing/2 - lambdas[0],0),
                direction='+',
                pol_angle=np.pi/2,
                name='planewave',
                )

        sim = sim.copy(update={'sources':[source]})

        time_monitorFieldOut = td.FieldTimeMonitor(
                    center = (0,0,0),
                    size = (Lx,Ly,0),
                        start=0,
                        stop=structure_1.t_stop,
                        interval=50,
                        fields=["Ex", "Ey", "Ez"],
                        name="time_monitorFieldOut",

                    )


        freq_monitorFieldOut = td.FieldMonitor(
                    center = (0,0,0),
                    size = (99, 25,0),
                       freqs =structure_1.monitor_freqs,
                        fields=["Ex", "Ey", "Ez", "Hx", "Hy", "Hz"],
                        name="freq_monitorFieldOut",

                    )


        time_monitorT = td.FluxTimeMonitor(
                        center = (
                                0,(Lx*slicing)/2+3,0
                                ),
                    size = (
                       td.inf,0,td.inf
                        ),
                        interval = 200,
                        name="time_monitorT",

                    )
        monitor_1 = td.FluxMonitor(
                center = (
                                0,-((Lx*slicing)/2+3),0
                                ),
                size = (
                   td.inf,0,td.inf
                    ),
                freqs = structure_1.monitor_freqs,
                name='flux2' )

        monitor_2 = td.FluxMonitor(
                center = (
                                0,(Lx*slicing)/2+3,0
                                ),
                size = (
                   td.inf,0,td.inf
                    ),
                freqs = structure_1.monitor_freqs,
                name='flux1' 
            )

        sim = sim.copy(update={'monitors':[freq_monitorFieldOut,monitor_1,monitor_2]})

        slab_1 = td.Structure(
                        geometry=td.Box(
                            center= (0,(Ly/2+Lx*slicing/2)/2,0),
                            size=(td.inf,(Ly/2-Lx*slicing/2),td.inf),
                        ),

                        medium=medium,
                        name='slab1',
                        )

        slab_2 = td.Structure(
                        geometry=td.Box(
                            center= (0,-(Ly/2+Lx*slicing/2)/2,0),
                            size=(td.inf,(Ly/2-Lx*slicing/2),td.inf),
                        ),
                        medium=medium,
                        name='slab2',
                        )

        cyl_group = []
        for x,y in data[k]['centers']:
            if np.abs(y)<=(Lx/2)*slicing:
                cyl_group.append(td.Cylinder(center=[x, y, 0], radius=0.189, length=td.inf))

        cylinders = td.Structure(geometry=td.GeometryGroup(geometries=cyl_group), medium=medium)


        if empty:
            sim = sim.copy(update={'structures':[],"grid_spec": td.GridSpec.uniform(dl=structure_1.dl)})
        else:
            sim = sim.copy(update={'structures':[cylinders],"grid_spec": td.GridSpec.uniform(dl=structure_1.dl)})
        fig, ax = plt.subplots(1, tight_layout=True, figsize=(16, 8))
        sim.plot(z=0, ax=ax)
        plt.show()

        sim_name = run_name
        if run:
            if add_ref:
                 id0 =web.upload(sim.copy(update={'structures':[]}), folder_name=project_name,task_name=fr"{sim_name}_0", verbose=True)
                 web.start(task_id = id0)
                 web.monitor(task_id=id0,verbose=True)
                 add_ref = False

            id =web.upload(sim, folder_name=project_name,task_name=sim_name, verbose=True)
            web.start(task_id = id)
            web.monitor(task_id=id,verbose=True)
            ids = id0+ '\n' + id
            incidence_folder = Path(file).stem
            file_path = rf"H:\phd stuff\tidy3d\data/{project_name}/{incidence_folder}/{sim_name}.txt"
            # Check if the folder exists
            if not os.path.exists( fr"H:\phd stuff\tidy3d\data/{project_name}/{incidence_folder}"):
                os.makedirs(fr"H:\phd stuff\tidy3d\data/{project_name}/{incidence_folder}")
                print(fr"Folder 'H:\phd stuff\tidy3d\data/{project_name}/{incidence_folder}' created successfully.")

            # Open file in write mode
            with open(file_path, "w") as f:
                # Write the string to the file
                f.write(ids)
        else:
            sim.plot_3d()
            id =web.upload(sim,task_name=sim_name, verbose=True)
            print(web.estimate_cost(id))
            raise TypeError("Program ends here")

